<a href="https://colab.research.google.com/github/knh0503/CrimeManagementSystem/blob/main/offender_predict_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 범인 예측 시스템 개요

데이터베이스에는 범죄자들의 인적사항이 기록되어 있다.

범죄를 저지르고 도망친 용의자를 추적하기 위햐여 데이터베이스를 활용하여 인적사항과 위치기록을 대조하여 용의자의 위치를 예측하려고 한다.




# 데이터베이스 수정 및 추가


우선 데이터베이스에 offender_descriptioin 테이블에는 다음 정보들이 있어야 한다.

[id, 가해자 id, 나이, 성별, 키, 체중, 범죄 위치, 과거 체포 기록]

# region 테이블에 위치, 경도 추가

region은 0~77까지의 숫자로 이루어져 있다. 이에 위치, 경도를 추가하여 머신러닝 학습에 필요한 특성을 늘린다.

In [12]:
import psycopg2
import random

conn = psycopg2.connect(
    dbname="crimemanagementsystem",
    user="dbproject",
    password="1234",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()

In [3]:
# 1. region 테이블에 latitude와 longitude 컬럼 추가
cursor.execute("""
    ALTER TABLE region
    ADD COLUMN IF NOT EXISTS latitude FLOAT,
    ADD COLUMN IF NOT EXISTS longitude FLOAT;
""")
conn.commit()

In [ ]:
cursor.execute("""
    update region
    set latitude = random() * (90 - (-90)) + (-90),
        longtitude = random() * (180 - (-180)) + (-180); 
""")
conn.commit

# offender_description 테이블 추가

In [4]:
# 1. offender_description 테이블 생성
cursor.execute("""
    CREATE TABLE offender_description (
        offender_id SERIAL PRIMARY KEY,
        age INT,
        gender bpchar(1),
        height FLOAT,
        weight FLOAT,
        latitude FLOAT,
        longitude FLOAT,
        previous_lat FLOAT[],
        previous_long FLOAT[]
    );
""")
conn.commit()

In [ ]:
# 2. 기존 offender 테이블 데이터 가져오기
cursor.execute("SELECT id, age, gender FROM offender")
offenders = cursor.fetchall()

In [ ]:
# 3. 각 범죄자 데이터 처리 및 offender_description에 삽입
for offender_id, age, gender in offenders:
    # 성별에 따른 키와 몸무게 랜덤 생성
    if gender == 'M':
        height = random.uniform(175, 195)  # 남성 키 범위
        weight = random.uniform(65, 95)   # 남성 몸무게 범위
    elif gender == 'F':
        height = random.uniform(150, 170)  # 여성 키 범위
        weight = random.uniform(40, 75)    # 여성 몸무게 범위
    else:
        height = None
        weight = None
    
    # offender_location에서 region 정보를 통해 latitude, longitude 설정
    cursor.execute(f"""
        SELECT r.latitude, r.longitude
        FROM offender_location ol
        JOIN region r ON ol.region_id = r.id
        WHERE ol.offender_id = {offender_id}
        LIMIT 1;
    """)
    location = cursor.fetchone()
    if location:
        latitude, longitude = location
    else:
        latitude = longitude = None

    # 과거 체포 위치 랜덤 생성 (1~5개)
    num_previous = random.randint(1, 5)
    previous_lat = [random.uniform(-90, 90) for _ in range(num_previous)]
    previous_long = [random.uniform(-180, 180) for _ in range(num_previous)]

    # offender_description 테이블에 데이터 삽입
    cursor.execute("""
        INSERT INTO offender_description (offender_id, age, gender, height, weight, latitude, longitude, previous_lat, previous_long)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);
    """, (offender_id, age, gender, height, weight, latitude, longitude, previous_lat, previous_long))

# 변경 사항 저장 및 연결 종료
conn.commit()
cursor.close()
conn.close()

print("offender_description 테이블 생성 및 데이터 삽입 완료.")

offender_description 테이블 생성 및 데이터 삽입 완료.


# longtitude 오류

테이블을 확인해 보니 longtitude가 제대로 삽입되지 않았다.
    location = cursor.fetchone()
    if location:
        latitude, longitude = location
    else:
        latitude = longitude = None

이 부분에서 latitude, longtitude = location[0], location[1]이 되어야 하는데 longtitude 값이 무시된 것으로 보인다.
따라서 offender_description 테이블의 latitude와 region 테이블의 latitude가 같은 데이터에서 longtitude를 가져와 붙여준다.

In [13]:
cursor.execute(f"""
UPDATE offender_description od
SET longitude = r.longitude
FROM region r
WHERE od.latitude = r.latitude;
""")

In [14]:
conn.commit()
cursor.close()
conn.close()

여전히 제대로 경도가 입력되지 않았다. 알고보니 longtitude로 잘못 입력했다.